In [8]:
from pathlib import Path
import os

PROJECT_ROOT = Path().resolve().parents[0]
print(PROJECT_ROOT)

/mnt/d/resources/cv projects/251219 branch spotter


In [11]:
import cv2
import numpy as np

mask = cv2.imread(PROJECT_ROOT / "./data/raw/rugd/annotations/creek/creek_00001.png", cv2.IMREAD_UNCHANGED)
print(mask.shape)
print(np.unique(mask.reshape(-1, mask.shape[-1] if mask.ndim == 3 else 1), axis=0))

(550, 688, 3)
[[  0 102   0]
 [  0 102 102]
 [  0 255   0]
 [  0 255 255]
 [ 20  64 108]
 [127   0 255]
 [204 153 255]
 [204 229 255]
 [255   0   0]
 [255 128   0]
 [255 204 153]
 [255 255 102]]


In [14]:
TARGET_COLORS = [
    (0, 255, 0),        # tree
    (255, 153, 204),    # bush
]

In [15]:
def convert_mask_color(mask_path, out_path):
    mask = cv2.imread(str(mask_path))  # BGR!

    binary = np.zeros(mask.shape[:2], dtype=np.uint8)

    for color in TARGET_COLORS:
        color_bgr = tuple(reversed(color))
        binary |= np.all(mask == color_bgr, axis=-1)

    binary = binary.astype(np.uint8) * 255
    cv2.imwrite(str(out_path), binary)

In [17]:
SRC_ANN = Path(PROJECT_ROOT / "data/raw/rugd/annotations")
DST_ANN = Path(PROJECT_ROOT / "data/processed/rugd/binary_annotations")

DST_ANN.mkdir(parents=True, exist_ok=True)

for video_dir in SRC_ANN.iterdir():
    if not video_dir.is_dir():
        continue

    out_video_dir = DST_ANN / video_dir.name
    out_video_dir.mkdir(exist_ok=True)

    for mask_path in video_dir.glob("*.png"):
        out_path = out_video_dir / mask_path.name
        convert_mask_color(mask_path, out_path)